In [1]:
from supporting_functions import *
from cryptanalysis import *
from constants import *
import dill

In [2]:
masked_plain_texts = []
masked_cipher_texts = []
for b in range(8):
    plain_texts = []
    for i in range(128):
        p = "ff"*(7-b)
        p += int_to_str(i)
        p += "ff"*(b)
        plain_texts.append(p)
    cipher_texts = encode_multiple(plain_texts)
    masked_plain_texts.append(plain_texts)
    masked_cipher_texts.append(cipher_texts)
masked_plain_texts.reverse()
masked_cipher_texts.reverse()

In [3]:
E = [25, 19, 48, 93, 74, 39, 118, 22]
M_diag = [38, 27, 11, 112, 12, 43, 70, 84]
E.reverse()
M_diag.reverse()

In [4]:
M = []
for i in range(8):
    M.append([None]*8)
    for j in range(8):
        if i==j:
            M[i][j] = M_diag[i]
        else:
            M[i][j] = 0

In [5]:
for i in range(8):
    for j in range(8-i):
        r = 7 - j
        c = 7 - i - j
        
        start = c
        end = r
        
        plain_texts = masked_plain_texts[c]
        cipher_texts = masked_cipher_texts[c]
        
        count = 0
        ansb = -1
        for b in range(128):
            M[r][c] = b
            flag = True
            
            for k in range(len(plain_texts)):
                plain_text = plain_texts[k]
                cipher_text = cipher_texts[k]
                expected = pair_to_poly(cipher_text[2*r:2*(r+1)])
                
                myvec = []
                for l in range(8):
                    myvec.append(pair_to_poly(plain_text[2*l:2*(l+1)]))

                # E
                for l in range(8):
                    myvec[l] = power_poly(myvec[l], E[l])
                
                # A
                newvec = [None] * 8
                for p in range(8):
                    ans = int_to_poly(0)
                    for q in range(8):
                        temp = mult_poly(myvec[q], int_to_poly(M[p][q]))
                        ans = add_poly(ans, temp)
                    newvec[p] = ans
                myvec = newvec
                
                # E
                for l in range(8):
                    myvec[l] = power_poly(myvec[l], E[l])
                  
                # A
                newvec = [None] * 8
                for p in range(8):
                    ans = int_to_poly(0)
                    for q in range(8):
                        temp = mult_poly(myvec[q], int_to_poly(M[p][q]))
                        ans = add_poly(ans, temp)
                    newvec[p] = ans
                myvec = newvec
                
                # E
                for l in range(8):
                    myvec[l] = power_poly(myvec[l], E[l])
                    
                if myvec[r]!=expected:
                    flag = False
                    break
            
            if flag==True:
                ansb = b
                count+=1 
            
        if count!=1:
            print("Found not one", r, c, count)
        else:
            M[r][c] = ansb
            print(r,c,M[r][c])  
        

7 7 38
6 6 27
5 5 11
4 4 112
3 3 12
2 2 43
1 1 70
0 0 84
7 6 25
6 5 89
5 4 98
4 3 123
3 2 4
2 1 31
1 0 112
7 5 70
6 4 28
5 3 47
4 2 1
3 1 17
2 0 14
7 4 25
6 3 102
5 2 30
4 1 61
3 0 99
7 3 27
6 2 20
5 1 53
4 0 110
7 2 73
6 1 123
5 0 29
7 1 2
6 0 15
7 0 68


In [6]:
with open("A","wb") as f:
    dill.dump(M, f)